<a href="https://colab.research.google.com/github/hager2164/tasks_follow/blob/main/Naive_Bayes_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Pyspark


In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()


In [3]:
spark

In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
iris = iris.frame
iris = spark.createDataFrame(iris)

In [5]:
iris.show()


+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

In [6]:
iris.printSchema()


root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- target: long (nullable = true)



In [7]:
from pyspark.ml.feature import VectorAssembler


In [8]:
featureassembler= VectorAssembler(inputCols=['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)'], outputCol='Features')

In [9]:
output = featureassembler.transform(iris)


In [10]:
modeldata=output.select('Features','target')


In [11]:
modeldata.show()

+-----------------+------+
|         Features|target|
+-----------------+------+
|[5.1,3.5,1.4,0.2]|     0|
|[4.9,3.0,1.4,0.2]|     0|
|[4.7,3.2,1.3,0.2]|     0|
|[4.6,3.1,1.5,0.2]|     0|
|[5.0,3.6,1.4,0.2]|     0|
|[5.4,3.9,1.7,0.4]|     0|
|[4.6,3.4,1.4,0.3]|     0|
|[5.0,3.4,1.5,0.2]|     0|
|[4.4,2.9,1.4,0.2]|     0|
|[4.9,3.1,1.5,0.1]|     0|
|[5.4,3.7,1.5,0.2]|     0|
|[4.8,3.4,1.6,0.2]|     0|
|[4.8,3.0,1.4,0.1]|     0|
|[4.3,3.0,1.1,0.1]|     0|
|[5.8,4.0,1.2,0.2]|     0|
|[5.7,4.4,1.5,0.4]|     0|
|[5.4,3.9,1.3,0.4]|     0|
|[5.1,3.5,1.4,0.3]|     0|
|[5.7,3.8,1.7,0.3]|     0|
|[5.1,3.8,1.5,0.3]|     0|
+-----------------+------+
only showing top 20 rows



In [12]:
train_data,test_data=modeldata.randomSplit([0.8,0.2])


In [13]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='Features', labelCol='target')
nb=nb.fit(train_data)



In [14]:
y_pred = nb.transform(test_data)
y_pred.show()


+-----------------+------+--------------------+--------------------+----------+
|         Features|target|       rawPrediction|         probability|prediction|
+-----------------+------+--------------------+--------------------+----------+
|[4.9,3.6,1.4,0.1]|     0|[-11.528295483034...|[0.78942243659543...|       0.0|
|[5.0,3.2,1.2,0.2]|     0|[-11.144595784515...|[0.75322827770834...|       0.0|
|[5.1,3.5,1.4,0.3]|     0|[-12.292317582714...|[0.73632731626717...|       0.0|
|[5.4,3.4,1.5,0.4]|     0|[-12.954768414924...|[0.69321851128191...|       0.0|
|[5.4,3.9,1.7,0.4]|     0|[-13.883549276798...|[0.72363065980502...|       0.0|
|[6.0,2.2,4.0,1.0]|     1|[-19.083586743177...|[0.07646148405954...|       1.0|
|[6.1,2.9,4.7,1.4]|     1|[-22.725015614817...|[0.04164500711472...|       1.0|
|[6.5,2.8,4.6,1.5]|     1|[-23.073347419434...|[0.03998474903553...|       1.0|
|[6.9,3.1,4.9,1.5]|     1|[-24.261687528625...|[0.04149296556800...|       1.0|
|[5.0,2.3,3.3,1.0]|     1|[-17.133103900

In [15]:
from sklearn.metrics import confusion_matrix
pred=y_pred.select("prediction").collect()
orig=y_pred.select("target").collect()
print(confusion_matrix(orig, pred))

[[5 0 0]
 [0 7 1]
 [0 0 6]]


In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', predictionCol='prediction')

In [17]:
accuracy = evaluator.evaluate(y_pred)
accuracy

0.9476383265856949

In [19]:
spark.stop()